# latticedemo example

LATTICEDEMO self-running tutorial
demonstrates 
1. ELEMENT, family of ELEMENTS, sequence of ELEMENTS 
2. Lattice representation
3. Creating a lattice 
4. Creating and editing lattice files

Import AT libraries

In [ ]:
import numpy as np
from at import atpass
from at import elements

An element in Accelerator Toolbox is a Python object

The folowing code creates a structure `d1` for a drift space
and a structure `qf` for a quadrupole.

In [ ]:
d1 = elements.Drift('DR01', 3)

qf = elements.Quadrupole('QF', 1, 0.2,
                         MaxOrder=3,
                         NumIntSteps=1,
                         PolynomA=np.zeros(4),
                         PolynomB=np.array([0, 0.2, 0, 0]),
                         R1=np.eye(6),
                         R2=np.eye(6),
                         T1=np.zeros(6),
                         T2=np.zeros(6))

Use `print` to print the element's info or access attributes using dot notation:

In [ ]:
display(d1)
print('-'*30)
print(d1)
print('-'*30)
print(f'FamName : {d1.FamName}')
print(f'Length : {d1.Length}')
print(f'PassMethod : {d1.PassMethod}')

print('='*30)

print(qf)
print('-'*30)
for attr in qf.REQUIRED_ATTRIBUTES:
    print(f'{attr} : {getattr(qf, attr)}')

The next few lines will create another drift structure `d2` from the exiting `d1`
and modify the values of fields `FamName` and `Length`

In [ ]:
d2 = d1.copy()
print(d1)
d2.FamName = 'DR02'
d2.Length = 2
print('-'*30)
print(d1)
print('-'*30)
print(d2)


Note that assignment operator does not create a copy of an element. In the following code `d3` is an alias of `d1`, pointing to the same underlying object.

In [ ]:
d3 = d1
print(d1)
d3.Length = 1
print('-'*30)
print(d1)

Create another quadrupole element structure `QD` from `QF` and modify
the values of fields `K` and `PolynomB` to make it defocusing 

In [ ]:
qd = qf.deepcopy()
qd.FamName = 'QD'
qd.K = -0.2
print(qd)

Field `PolynomB` is a vector with polynomial field expansion coefficients.
The second element (quadrupole coefficient) must be consistent with field `K`. Note that pyAT took care of that.

**Rule of thumb** If you're not sure if you should use `copy` or `deepcopy`, just stick with `deepcopy`.

We have declared four elements:

In [ ]:
display(d1)
display(d2)
display(qf)
display(qd)

They are now independent from each other

We are ultimately interested in sequences of elements
to model storage ring lattices or single-pass beam transport lines.
The next section will illustrate building of such sequences

Accelerator Toolbox represents sequences of elements as Python iterable object, in this case `numpy.array`.
The following commad creates a simple `FODO` cell by copying previously 
created element structures for drifts and quadrupole magnets to a lattce list `fodocell`:

In [ ]:
fodocell = np.array([d1, qf, d2, qd])
fodocell

`len` is useful to find the number of elements in a sequence

In [ ]:
print(len(fodocell))
print(fodocell.size)

Use `[:]` list syntax to print some elements

In [ ]:
display(fodocell[1])
print('-'*30)
display(fodocell[2:])

`numpy.array` allows slicing by list

In [ ]:
display(fodocell[[0,2]])

Let's build a lattice `thering` that represents a closed ring 
with 10 periods of `fodocell`

In [ ]:
from at.lattice import Lattice

In [ ]:
thering = Lattice(fodocell, name='THERING', periodicity=10, energy=1e6)

We can visualy examine the ring period

In [ ]:
from at.plot import plot_synopt

In [ ]:
plot_synopt(thering)

Lattice `thering` is a variable in Octave workspace.
We can use it in accelerator physics functions and scripts

For example: function `FindM44` finds 4-by-4 transverse transfer matrix

In [ ]:
at.find_m44(thering)

Help pages for functions may be viewed in Jupyter Web IDE

In [ ]:
?at.find_m44